In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from datasets.img_dataset import FoodImgs
from models.cnn import SnapSnack

In [2]:
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
import math


dataset = FoodImgs()

s = len(dataset)
print(f"{s} imgs")
train_size = int(math.ceil(s * 0.8))
test_size = s - train_size

train_set, test_set = random_split(dataset, [train_size, test_size])


9090 imgs


In [27]:
import torch
model = SnapSnack(fc_layers=(), output_dim=4, lr=1)
model.load_state_dict(torch.load("../models/model_10.pt", map_location=torch.device("cpu")))

<All keys matched successfully>

In [28]:
model = model.cpu().eval()

In [43]:
from tqdm.notebook import tqdm
preds = []
targets = []
from datasets.img_dataset import UnNormalize
unnorm = UnNormalize()
with torch.no_grad():
    for img, target in tqdm(DataLoader(test_set, batch_size=1)):
        pred = model(img)
        preds.append(unnorm(pred))
        targets.append(unnorm(target))

In [44]:
from sklearn.metrics import r2_score
import numpy as np
if type(preds) is list:
    preds = torch.cat(preds)
    targets = torch.cat(targets)
r2_score(np.abs(targets.numpy()), np.abs(preds.numpy()))
# r2_score(targets.numpy(), preds.numpy())

-0.0028166149505752247

In [46]:
preds.min(), targets.min()

(tensor(2.7274), tensor(0.))

In [47]:
preds

tensor([[437.0171,  16.0986,  26.0172, 695.4551],
        [561.7183,  20.3758,  32.9166, 572.1035],
        [595.2773,  21.1195,  35.2029, 551.6035],
        ...,
        [549.6860,  19.5089,  31.6379, 627.0562],
        [503.7739,  18.1613,  26.1581, 720.0278],
        [516.8618,  19.2107,  26.8328, 665.7134]])